# Notebook 01 - Building 1st and 2nd floor data cleaning and fixing

## Objectives
* Clean data
* Evaluate and process missing data
* Fix potential issues with data in features (1stFlrSF and 2ndFlrSF)

## Inputs
* outputs/datasets/collection/HousePricesRecords.csv

## Outputs
* Clean and fix (missing and potentially wrong) data in given column
* After cleaning is completed, we will save current dataset in inputs/datasets/cleaning/floors.csv

## Change working directory
In This section we will get location of current directory and move one step up, to parent folder, so App will be accessing project folder.

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/Users/pecukevicius/DataspellProjects/heritage_houses_p5/jupyter_notebooks/data_cleaning'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chdir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("you have set a new current directory")

you have set a new current directory


Confirm new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/Users/pecukevicius/DataspellProjects/heritage_houses_p5/jupyter_notebooks'

We need to check current working directory

In [4]:
current_dir

'/Users/pecukevicius/DataspellProjects/heritage_houses_p5/jupyter_notebooks'

We can see that current is **jupyter_notebooks**, as current notebook is in subfolder. We will go one step up to parent directory, what will be our project main directory.
Print out to confirm working directory

In [5]:
os.chdir(os.path.dirname(current_dir))
current_dir = os.getcwd()
current_dir

'/Users/pecukevicius/DataspellProjects/heritage_houses_p5'

## Loading Dataset

In [6]:
import pandas as pd

df = pd.read_csv("outputs/datasets/collection/HousePricesRecords.csv")
df.head()

,Unnamed: 0,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,0,856,854.0,3.0,No,706,GLQ,150,0.0,548,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,...,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,...,68.0,162.0,42,5,7,920,NaN,2001,2002,223500
3,3,961,NaN,NaN,No,216,ALQ,540,NaN,642,...,60.0,0.0,35,5,7,756,NaN,1915,1970,140000
4,4,1145,NaN,4.0,Av,655,GLQ,490,0.0,836,...,84.0,350.0,84,5,8,1145,NaN,2000,2000,250000


## Exploring Data

We will get all features that are missing data as a list

### Checking if there is any missing values in given features (1st and 2nd floor areas)

First we will check features for missing values

In [7]:
# List of column names to check for missing values
features_to_check = ['1stFlrSF', '2ndFlrSF']

# Loop through each column in the list
for column in features_to_check:
    # Check for missing values
    if df[column].isna().sum() > 0:
        print(f"There are missing values in '{column}'.")
        # Fill missing values with a default value - 0
        df[column] = df[column].fillna(0)
    else:
        print(f"No missing values in '{column}'.")


No missing values in '1stFlrSF'.
There are missing values in '2ndFlrSF'.


### Checking Data Type

In [8]:
df[['1stFlrSF', '2ndFlrSF']].dtypes

1stFlrSF      int64
2ndFlrSF    float64
dtype: object

2ndFlrSF is float, we need to convert to integer

In [9]:
df['2ndFlrSF'] = df['2ndFlrSF'].astype(int)

### Checking Values for lower than zero

In [10]:
# Loop through each column in the list
for column in features_to_check:
    # Check if there are any negative values in the column
    if (df[column] < 0).any():
        print(f"There are negative values in '{column}', which is not allowed.")
    else:
        print(f"No negative values in '{column}'.")


No negative values in '1stFlrSF'.
No negative values in '2ndFlrSF'.


### Checking for wrong data values, where 1st or 2nd floor area is bigger than ground-floor

In [11]:
for floor_col in features_to_check:
    # Find entries where floor area exceeds 'GrLivArea'
    invalid_areas = df[df[floor_col] > df['GrLivArea']]
    if not invalid_areas.empty:
        print(f"There are entries where '{floor_col}' is greater than 'GrLivArea'.")
        # Display the problematic entries
        print(invalid_areas[[floor_col, 'GrLivArea']])
    else:
        print(f"All '{floor_col}' values are within the valid range of 'GrLivArea'.")


All '1stFlrSF' values are within the valid range of 'GrLivArea'.
All '2ndFlrSF' values are within the valid range of 'GrLivArea'.


We need also to inspect, is there any 2nd floor bigger then 1st

In [12]:
invalid_areas = df[df['1stFlrSF'] < df['2ndFlrSF']]
if not invalid_areas.empty:
    print(" There are records where 2nd floor is bigger than 1st floor, total number of records: ", invalid_areas.shape[0])
else:
    print("All values are correct")

 There are records where 2nd floor is bigger than 1st floor, total number of records:  129


We have found 129 records, where 2nd floor is bigger than 1st.

Such data is very unlikely.
Our steps:
1. Create extra column in dataset - to store which records are wrong
2. Create a copy of current dataset
3. Filter dataset where 2nd floor is bigger than 1st floor

In [13]:
# Creating extra column in dataset to store where 2nd floor is bigger than 1st
df['2nd_floor_larger'] = df['1stFlrSF'] < df['2ndFlrSF']

# Filtering dataset for wrong records and making copy of such dataset
bad_records = df[df['2nd_floor_larger']].copy()

# Calculating ratios
bad_records['floor_ratio'] = ((bad_records['2ndFlrSF'] - bad_records['1stFlrSF'])/bad_records['1stFlrSF']) * 100

bad_records[['1stFlrSF', '2ndFlrSF', 'floor_ratio']]

,1stFlrSF,2ndFlrSF,floor_ratio
20,1158,1218,5.181347
30,649,668,2.927581
35,1132,1320,16.607774
58,1426,1519,6.521739
65,1143,1330,16.360455
...,...,...,...
1365,814,860,5.651106
1366,882,908,2.947846
1378,483,504,4.347826
1395,1133,1349,19.064431


We can see there are quite high differences. Let's check what is average, just for curiosity

In [14]:
bad_records['floor_ratio'].mean()

14.76282609025632

This is sad, as average is so high, and there were many possibilities for such abnormalities to happen:
* mistyping
* entering values in wrong cells - 1st and 2nf floor areas were swapped when entering data
* It is real to be 2nd floor bigger, but it is very unlikely and uncommon, so we reject this

Let's check, is there any records, where 2nd floor is greater than 1st floor

In [15]:
test = bad_records[bad_records['2ndFlrSF'] > bad_records['GrLivArea']]
test

,Unnamed: 0,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,...,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice,2nd_floor_larger,floor_ratio


We can see, after given test, there is no records where swapped, and instead of entering to 1st floor, it was entered to 2nd floor

We will swap those given values back

In [16]:
indexes = df['2ndFlrSF'] > df['1stFlrSF']
df.loc[indexes, ['1stFlrSF', '2ndFlrSF']] = df.loc[indexes, ['2ndFlrSF', '1stFlrSF']].values

Let's check again, is there any records where 2nd floor is bigger than 1st, just to check if all is fixed

In [17]:
# Creating extra column in dataset to store where 2nd floor is bigger than 1st
df['2nd_floor_larger'] = df['1stFlrSF'] < df['2ndFlrSF']

# Filtering dataset for wrong records and making copy of such dataset
bad_records = df[df['2nd_floor_larger']].copy()

# Calculating ratios
bad_records['floor_ratio'] = ((bad_records['2ndFlrSF'] - bad_records['1stFlrSF'])/bad_records['1stFlrSF']) * 100

bad_records[['1stFlrSF', '2ndFlrSF', 'floor_ratio']]

,1stFlrSF,2ndFlrSF,floor_ratio


We can see all records are fixed now.

As we have created extra columns in given dataset, before exporting it as csv, we will remove them, so amount of features will remain the same

In [18]:
# Importing original dataset
df_original = pd.read_csv('outputs/datasets/collection/HousePricesRecords.csv')

# Identify features that are in current and original datasets
matching_features = df.columns.intersection(df_original.columns)

# Applying just existing features, remaining will be discarded
df = df[matching_features]

df.head()

,Unnamed: 0,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,0,856,854,3.0,No,706,GLQ,150,0.0,548,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1,1262,0,3.0,Gd,978,ALQ,284,NaN,460,...,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,2,920,866,3.0,Mn,486,GLQ,434,0.0,608,...,68.0,162.0,42,5,7,920,NaN,2001,2002,223500
3,3,961,0,NaN,No,216,ALQ,540,NaN,642,...,60.0,0.0,35,5,7,756,NaN,1915,1970,140000
4,4,1145,0,4.0,Av,655,GLQ,490,0.0,836,...,84.0,350.0,84,5,8,1145,NaN,2000,2000,250000


## Exporting current dataset as csv

In [19]:
df.to_csv('inputs/datasets/cleaning/floors.csv', index=False)

### Adding code to cleaning Pipeline:

```python
import pandas as pd

def fill_missing_values(df, columns, fill_value=0):
    """
    Fills missing values in specified columns with a given fill value.
    
    Args:
    df (DataFrame): The pandas DataFrame to modify.
    columns (list of str): List of column names to check for missing values.
    fill_value (int, optional): The value to use when filling missing values. Defaults to 0.
    """
    for column in columns:
        missing_count = df[column].isna().sum()
        if missing_count > 0:
            print(f"There are missing values in '{column}'.")
            df[column].fillna(fill_value, inplace=True)
        else:
            print(f"No missing values in '{column}'.")

def analyze_floor_differences(df):
    """
    Analyzes differences between the 1st and 2nd floors, creates new columns,
    and filters data based on specific conditions.

    Args:
    df (DataFrame): The pandas DataFrame to analyze.

    Returns:
    DataFrame: A DataFrame containing bad records with additional analysis.
    """
    df['2ndFlrSF'] = df['2ndFlrSF'].astype(int)
    df['2nd_floor_larger'] = df['1stFlrSF'] < df['2ndFlrSF']
    bad_records = df[df['2nd_floor_larger']].copy()
    bad_records['floor_ratio'] = ((bad_records['2ndFlrSF'] - bad_records['1stFlrSF']) / bad_records['1stFlrSF']) * 100
    print("Average floor ratio where 2nd floor is larger:", bad_records['floor_ratio'].mean())
    
    # Swap floors if the second floor area is greater than the first
    indexes = df['2ndFlrSF'] > df['1stFlrSF']
    df.loc[indexes, ['1stFlrSF', '2ndFlrSF']] = df.loc[indexes, ['2ndFlrSF', '1stFlrSF']].values
    return bad_records

def synchronize_datasets(df, dataset_path):
    """
    Loads an original dataset and synchronizes the features with the current DataFrame.

    Args:
    df (DataFrame): The current pandas DataFrame.
    dataset_path (str): Path to the original dataset CSV file.

    Returns:
    DataFrame: The synchronized DataFrame with matching features only.
    """
    df_original = pd.read_csv(dataset_path)
    matching_features = df.columns.intersection(df_original.columns)
    return df[matching_features]

# Fill missing values for specified features
features_to_check = ['1stFlrSF', '2ndFlrSF']
fill_missing_values(df, features_to_check)

# Analyze and adjust records based on floor areas
bad_records = analyze_floor_differences(df)
print(bad_records[['1stFlrSF', '2ndFlrSF', 'floor_ratio']])

# Synchronize with an original dataset and display the first few rows
df = synchronize_datasets(df, 'outputs/datasets/collection/HousePricesRecords.csv')
print(df.head())
```

## Next step is cleaning and fixing Bedrooms